# Chap04 分组

In [1]:
import numpy as np
import pandas as pd

## 分组模式及其对象
### 分组的一般模式
1. 明确分组的三个要素：**分组依据、数据来源、操作及其返回结果**
   - `df.groupby(分组依据)[数据来源].使用操作`
### 分组依据的本质
1. groupby的分组依据除了可以直接从列中按照名字获取，也可以通过一定的复杂逻辑分组
### groupby对象
1. 通过`gb.ngroups`属性，得到分组个数
2. 通过`gb.groups.keys()`属性，可以返回从组名映射到组索引列表的字典
3. 通过`gb.size()`属性，统计每个组的元素个数
4. 通过`gb.get_group(组名元组)`方法可以直接获取所在组对应行，此时必须知道组的具体名字
### 分组的三大操作
1. 聚合`agg` eg.依据性别分组，统计全国人口寿命的平均值
2. 变换`transform` eg.依据季节分组，对每一个季节的温度进行组内标准化
3. 过滤`filter` eg.依据班级筛选出组内数学分数的平均值超过80分的班级

In [2]:
df = pd.read_csv('./data/learn_pandas.csv')

In [5]:
condition = df.Weight > df.Weight.mean()
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

In [11]:
item = np.random.choice(list('abc'), df.shape[0])
df.groupby(item)['Height'].mean()

a    164.244643
b    162.135484
c    163.366154
Name: Height, dtype: float64

## 聚合函数
### 内置聚合函数
1. max/min/mean/median/count/all/any/idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod
2. 性能相对较好
3. 缺点
   - 无法同时使用多个函数
   - 无法对特定的列使用特定的聚合函数
   - 无法使用自定义的聚合函数
   - 无法直接对结果的列名在聚合前进行自定义命名
### agg方法
1. 使用多个函数`gb.agg(聚合函数列表)`
2. 对特定的列使用特定的聚合函数`gb.agg(字典{列名:聚合函数or聚合函数列表})`
3. 在agg中可以使用具体的自定义函数
   - 由于传入的是序列，因此序列上的方法和属性都可以在函数中使用，只需保证返回值是标量即可
4. 聚合结果重命名，将函数的位置改写成元组`(新名字,函数原名)`
   - 对**一个或者多个列***使用**单个聚合**的时候，重命名需要加方括号`gb.agg([('my_sum', 'sum)])`

In [14]:
gb = df.groupby('Gender')[['Height','Weight']]
gb.agg(lambda x: x.mean()-x.min())

,Height,Weight
Gender,,
Female,13.79697,13.918519
Male,17.92549,21.759259


In [15]:
def my_func(s):
    res = 'High'
    if s.mean() <= df[s.name].mean():
        res = 'Low'
    return res
gb.agg(my_func)

,Height,Weight
Gender,,
Female,Low,Low
Male,High,High


In [16]:
gb.agg([('range', lambda x:x.max()-x.min()), ('my_sum','sum')])

Height          Weight        
        range   my_sum  range  my_sum
Gender                               
Female   24.8  21014.0   29.0  6469.0
Male     38.2   8854.9   38.0  3929.0